# Trực quan hóa dữ liệu

## Mục tiêu:

## Bước 1 - Đọc dữ liệu

Import các thư viện cần thiết

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

Đọc dữ liệu từ file CSV đã được tiền xử lý

In [4]:
# Đọc dữ liệu
df = pd.read_csv("../data/processed/processed_data.csv")
print("=== Dữ liệu đã qua tiền xử lý (5 dòng đầu) ===")
df.head()  

=== Dữ liệu đã qua tiền xử lý (5 dòng đầu) ===


,country_name,country_code,year,population,poverty_ratio,pop_growth,life_expectancy,gdp_per_capita,gdp_growth,sanitation,electricity,water_access,co2_emissions,slum_population,labor_force
0,Aruba,ABW,2000,90588.0,6.7980,1.030817,72.939,20681.023027,7.622921,30.949145,91.7,94.488806,2.965073,0.000000,60.895221
1,Aruba,ABW,2001,91439.0,1.9710,0.935033,73.044,20740.132583,4.182002,55.384827,100.0,94.770491,2.969193,6.662656,60.895221
2,Aruba,ABW,2002,92074.0,3.1740,0.692052,73.135,21307.248251,-0.944953,57.927596,100.0,95.052176,3.221322,0.000000,60.895221
3,Aruba,ABW,2003,93128.0,1.6095,1.138229,73.236,21949.485996,1.110505,56.465393,100.0,95.333861,3.671291,4.119222,60.895221
4,Aruba,ABW,2004,95138.0,4.8100,2.135358,73.223,23700.631990,7.293728,59.253413,100.0,95.615545,3.667304,0.000000,60.895221


## Bước 2: Tổng quan dữ liệu

Phân tích cấu trúc dữ liệu, kiểu dữ liệu và ý nghĩa của từng thuộc tính. 

In [5]:
print("THÔNG TIN TỔNG QUAN VỀ DỮ LIỆU:")
print(f"Kích thước dữ liệu: {df.shape}")
print(f"Số quốc gia: {df['country_name'].nunique()}")
print(f"Khoảng thời gian: {df['year'].min()}-{df['year'].max()}")

print("\nÝ NGHĨA CÁC THUỘC TÍNH:")
print("1. country_name: Tên quốc gia")
print("2. country_code: Mã quốc gia")
print("3. year: Năm")
print("4. population: Tổng dân số")
print("5. poverty_ratio: Tỷ lệ nghèo (% dân số)")
print("6. pop_growth: Tăng trưởng dân số hàng năm (%)")
print("7. life_expectancy: Tuổi thọ trung bình (năm) - TARGET")
print("8. gdp_per_capita: GDP bình quân đầu người (USD)")
print("9. gdp_growth: Tăng trưởng GDP hàng năm (%)")
print("10. sanitation: Tỷ lệ sử dụng dịch vụ vệ sinh an toàn (%)")
print("11. electricity: Tỷ lệ tiếp cận điện (%)")
print("12. water_access: Tỷ lệ sử dụng nước uống cơ bản (%)")
print("13. co2_emissions: Lượng khí thải CO2 bình quân đầu người (tấn)")
print("14. slum_population: Tỷ lệ dân số sống trong khu ổ chuột (%)")
print("15. labor_force: Tỷ lệ tham gia lực lượng lao động (%)")

THÔNG TIN TỔNG QUAN VỀ DỮ LIỆU:
Kích thước dữ liệu: (5425, 15)
Số quốc gia: 217
Khoảng thời gian: 2000-2024

Ý NGHĨA CÁC THUỘC TÍNH:
1. country_name: Tên quốc gia
2. country_code: Mã quốc gia
3. year: Năm
4. population: Tổng dân số
5. poverty_ratio: Tỷ lệ nghèo (% dân số)
6. pop_growth: Tăng trưởng dân số hàng năm (%)
7. life_expectancy: Tuổi thọ trung bình (năm) - TARGET
8. gdp_per_capita: GDP bình quân đầu người (USD)
9. gdp_growth: Tăng trưởng GDP hàng năm (%)
10. sanitation: Tỷ lệ sử dụng dịch vụ vệ sinh an toàn (%)
11. electricity: Tỷ lệ tiếp cận điện (%)
12. water_access: Tỷ lệ sử dụng nước uống cơ bản (%)
13. co2_emissions: Lượng khí thải CO2 bình quân đầu người (tấn)
14. slum_population: Tỷ lệ dân số sống trong khu ổ chuột (%)
15. labor_force: Tỷ lệ tham gia lực lượng lao động (%)
